<a href="https://colab.research.google.com/github/rickiepark/the-lm-book/blob/main/emotion_GPT2_as_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="display: flex; justify-content: center;">
    <div style="background-color: #f4f6f7; padding: 15px; width: 80%;">
        <table style="width: 100%">
            <tr>
                <td style="vertical-align: middle;">
                    <span style="font-size: 14px;">
                        A notebook for <a href="https://www.thelmbook.com" target="_blank" rel="noopener">The Hundred-Page Language Models Book</a> by Andriy Burkov<br><br>
                        Code repository: <a href="https://github.com/rickiepark/the-lm-book" target="_blank" rel="noopener">https://github.com/rickiepark/the-lm-book</a>
                    </span>
                </td>
                <td style="vertical-align: middle;">
                    <a href="https://www.thelmbook.com" target="_blank" rel="noopener">
                        <img src="https://thelmbook.com/img/book.png" width="80px" alt="The Hundred-Page Language Models Book">
                    </a>
                </td>
            </tr>
        </table>
    </div>
</div>

In [1]:
# 필수 라이브러리 임포트
import torch           # 메인 파이토치 라이브러리
from torch.utils.data import DataLoader  # 데이터셋 처리를 위한 라이브러리
from torch.optim import AdamW    # 훈련을 위한 옵티마이저
from transformers import AutoTokenizer, AutoModelForSequenceClassification  # 허깅 페이스 구성 요소
from tqdm import tqdm   # 진행률 표시줄 유틸리티
import json             # JSON 데이터 구문 분석을 위한 라이브러리
import requests         # URL에서 데이터셋 다운로드를 위한 라이브러리
import gzip             # 데이터셋 압축 해제를 위한 라이브러리
import random           # 시드 설정 및 데이터 셔플링을 위한 라이브러리

def set_seed(seed):
    """
    다양한 라이브러리에서 재현성을 위해 난수 시드를 설정합니다.
    Args:
        seed (int): 난수 생성을 위한 시드 값
    """
    # 파이썬 내장 random 시드 설정
    random.seed(seed)
    # 파이토치 CPU 난수 시드 설정
    torch.manual_seed(seed)
    # 사용 가능한 모든 GPU에 대한 시드 설정
    torch.cuda.manual_seed_all(seed)
    # cuDNN이 결정론적 알고리즘을 사용하도록 요청
    torch.backends.cudnn.deterministic = True
    # 일관된 동작을 위해 cuDNN의 자동 튜너 비활성화
    torch.backends.cudnn.benchmark = False

def load_and_split_dataset(url, test_ratio=0.1):
    """
    데이터셋을 다운로드하고 훈련 및 테스트 세트로 분할합니다.
    Args:
        url (str): 데이터셋의 URL
        test_ratio (float): 테스트를 위한 데이터 비율
    Returns:
        tuple: (train_dataset, test_dataset)
    """
    # 데이터셋 다운로드 및 압축 해제
    response = requests.get(url)
    content = gzip.decompress(response.content).decode()
    # JSON 라인을 예시 목록으로 구문 분석
    dataset = [json.loads(line) for line in content.splitlines()]
    # 데이터셋 셔플 및 분할
    random.shuffle(dataset)
    split_index = int(len(dataset) * (1 - test_ratio))
    return dataset[:split_index], dataset[split_index:]

def load_model_and_tokenizer(model_name, device, label_to_id, id_to_label, unique_labels):
    """
    시퀀스 분류를 위한 모델과 토크나이저를 로드하고 구성합니다.
    Args:
        model_name (str): 사전 훈련된 모델의 이름
        device: 모델을 로드할 장치
        label_to_id (dict): 레이블 문자열에서 ID로의 매핑
        id_to_label (dict): ID에서 레이블 문자열로의 매핑
        unique_labels (list): 모든 가능한 레이블의 목록
    Returns:
        tuple: (model, tokenizer)
    """
    # 올바른 수의 출력 클래스로 모델 초기화
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=len(unique_labels)
    )
    # 패딩 및 레이블 매핑 구성
    model.config.pad_token_id = model.config.eos_token_id
    model.config.id2label = id_to_label
    model.config.label2id = label_to_id
    # 토크나이저 초기화 및 구성
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    return (model.to(device), tokenizer)

def encode_text(tokenizer, text, return_tensor=False):
    """
    제공된 토크나이저를 사용하여 텍스트를 인코딩합니다.
    Args:
        tokenizer: 허깅 페이스 토크나이저
        text (str): 인코딩할 텍스트
        return_tensor (bool): 파이토치 텐서를 반환할지 여부
    Returns:
        토큰 ID의 목록 또는 텐서
    """
    # 텐서 출력이 요청된 경우, 파이토치 텐서로 인코딩
    if return_tensor:
        return tokenizer.encode(
            text, add_special_tokens=False, return_tensors="pt"
        )
    # 그렇지 않으면 토큰 ID 목록 반환
    else:
        return tokenizer.encode(text, add_special_tokens=False)

class TextClassificationDataset(torch.utils.data.Dataset):
    """
    텍스트 분류를 위한 파이토치 데이터셋.
    텍스트와 레이블을 모델 준비 형식으로 변환합니다.
    Args:
        data (list): 텍스트와 레이블을 포함하는 사전 목록
        tokenizer: 허깅 페이스 토크나이저
        label_to_id (dict): 레이블 문자열에서 ID로의 매핑
    """
    def __init__(self, data, tokenizer, label_to_id):
        self.data = data
        self.tokenizer = tokenizer
        self.label_to_id = label_to_id

    def __len__(self):
        # 총 예시 수 반환
        return len(self.data)

    def __getitem__(self, idx):
        """
        단일 훈련 예시를 반환합니다.
        Args:
            idx (int): 가져올 예시의 인덱스
        Returns:
            dict: input_ids와 labels를 포함
        """
        # 데이터셋에서 예시 가져오기
        item = self.data[idx]
        # 텍스트를 토큰 ID로 변환
        input_ids = encode_text(self.tokenizer, item["text"])
        # 레이블 문자열을 ID로 변환
        labels = self.label_to_id[item["label"]]
        return {
            "input_ids": input_ids,
            "labels": labels
        }

def collate_fn(batch):
    """
    예시 배치를 훈련 준비 형식으로 통합합니다.
    패딩과 텐서 변환을 처리합니다.
    Args:
        batch: 데이터셋의 예시 목록
    Returns:
        dict: input_ids, labels, attention_mask 텐서를 포함
    """
    # 패딩을 위해 가장 긴 시퀀스 찾기
    max_length = max(len(item["input_ids"]) for item in batch)
    # 0으로 입력 시퀀스 패딩
    input_ids = [
        item["input_ids"] +
        [0] * (max_length - len(item["input_ids"]))
        for item in batch
    ]
    # 어텐션 마스크 생성 (토큰은 1, 패딩은 0)
    attention_mask = [
        [1] * len(item["input_ids"]) +
        [0] * (max_length - len(item["input_ids"]))
        for item in batch
    ]
    # 레이블 수집
    labels = [item["labels"] for item in batch]
    # 모든 항목을 텐서로 변환
    return {
        "input_ids": torch.tensor(input_ids),
        "labels": torch.tensor(labels),
        "attention_mask": torch.tensor(attention_mask)
    }

def generate_label(model, tokenizer, text):
    """
    입력 텍스트에 대한 레이블 예측을 생성합니다.
    Args:
        model: 미세 튜닝된 모델
        tokenizer: 연관된 토크나이저
        text (str): 분류할 입력 텍스트
    Returns:
        str: 예측된 레이블
    """
    # 텍스트 인코딩 및 모델 장치로 이동
    input_ids = encode_text(
        tokenizer,
        text,
        return_tensor=True
    ).to(model.device)
    # 모델 예측 가져오기
    outputs = model(input_ids)
    logits = outputs.logits[0]
    # 가장 높은 확률의 클래스 가져오기
    predicted_class = logits.argmax().item()
    # 클래스 ID를 레이블 문자열로 변환
    return model.config.id2label[predicted_class]

def calculate_accuracy(model, dataloader):
    """
    데이터셋에서 예측 정확도를 계산합니다.
    Args:
        model: 미세 튜닝된 모델
        dataloader: 평가 예시를 포함하는 DataLoader
    Returns:
        float: 정확도 점수
    """
    # 모델을 평가 모드로 설정
    model.eval()
    correct = 0
    total = 0
    # 효율성을 위해 그레이디언트 계산 비활성화
    with torch.no_grad():
        for batch in dataloader:
            # 배치를 장치로 이동
            input_ids = batch["input_ids"].to(model.device)
            attention_mask = batch["attention_mask"].to(model.device)
            labels = batch["labels"].to(model.device)
            # 모델 예측 가져오기
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = outputs.logits.argmax(dim=-1)
            # 정확도 카운터 업데이트
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    # 정확도 계산
    accuracy = correct / total
    # 모델을 훈련 모드로 재설정
    model.train()
    return accuracy

def create_label_mappings(train_dataset):
    """
    레이블 문자열과 ID 사이의 매핑을 생성합니다.
    Args:
        train_dataset: 훈련 예시 목록
    Returns:
        tuple: (label_to_id, id_to_label, unique_labels)
    """
    # 정렬된 고유 레이블 목록 가져오기
    unique_labels = sorted(set(item["label"] for item in train_dataset))
    # 레이블과 ID 사이의 매핑 생성
    label_to_id = {label: i for i, label in enumerate(unique_labels)}
    id_to_label = {i: label for label, i in label_to_id.items()}
    return label_to_id, id_to_label, unique_labels

def test_model(model_path, test_input):
    """
    저장된 모델을 단일 입력으로 테스트합니다.
    Args:
        model_path (str): 저장된 모델의 경로
        test_input (str): 분류할 텍스트
    """
    # 장치 설정 및 모델 로드
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    # 예측 생성 및 표시
    emotion = generate_label(model, tokenizer, test_input)
    print(f"입력: {test_input}")
    print(f"예측된 감정: {emotion}")

def download_and_prepare_data(data_url, tokenizer, batch_size):
    """
    훈련을 위해 데이터셋을 다운로드하고 준비합니다.
    Args:
        data_url (str): 데이터셋의 URL
        tokenizer: 텍스트 처리를 위한 토크나이저
        batch_size (int): DataLoader의 배치 크기
    Returns:
        tuple: (train_dataloader, test_dataloader, label_to_id, id_to_label, unique_labels)
    """
    # 데이터셋 로드 및 분할
    train_dataset, test_dataset = load_and_split_dataset(data_url)
    # 레이블 매핑 생성
    label_to_id, id_to_label, unique_labels = create_label_mappings(train_dataset)
    # 데이터셋 생성
    train_data = TextClassificationDataset(
        train_dataset,
        tokenizer,
        label_to_id
    )
    test_data = TextClassificationDataset(
        test_dataset,
        tokenizer,
        label_to_id
    )
    # 데이터로더 생성
    train_dataloader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn
    )
    test_dataloader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn
    )
    return train_dataloader, test_dataloader, label_to_id, id_to_label, unique_labels

def get_hyperparameters():
    """
    훈련 하이퍼파라미터를 반환합니다.
    Returns:
        tuple: (num_epochs, batch_size, learning_rate)
    """
    # 시퀀스 분류가 더 빨리 수렴하므로 더 적은 에포크로 훈련
    num_epochs=8
    # 대부분의 GPU 메모리에서 잘 작동하는 표준 배치 크기
    batch_size=16
    # 트랜스포머 미세 튜닝을 위한 표준 학습률
    learning_rate=5e-5
    return num_epochs, batch_size, learning_rate

In [2]:
# 재현성을 위해 난수 시드 설정
seed = 42
set_seed(seed)

# 훈련 매개변수 구성
data_url = "https://www.thelmbook.com/data/emotions"
model_name = "openai-community/gpt2"

# 하이퍼파라미터 가져오기
num_epochs, batch_size, learning_rate = get_hyperparameters()

# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# 데이터 준비 및 레이블 매핑 가져오기
train_loader, test_loader, label_to_id, id_to_label, unique_labels = download_and_prepare_data(
    data_url,
    tokenizer,
    batch_size
)

# 시퀀스 분류를 위한 모델 초기화
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(unique_labels)
).to(device)

# 모델의 레이블 처리 구성
model.config.pad_token_id = model.config.eos_token_id
model.config.id2label = id_to_label
model.config.label2id = label_to_id

# 옵티마이저 초기화
optimizer = AdamW(model.parameters(), lr=learning_rate)

# 훈련 루프
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    num_batches = 0
    progress_bar = tqdm(train_loader, desc=f"에포크 {epoch+1}/{num_epochs}")

    for batch in progress_bar:
        # 배치를 장치로 이동
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # 포워드 패스
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        # 백워드 패스 및 최적화
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # 메트릭 업데이트
        total_loss += loss.item()
        num_batches += 1
        progress_bar.set_postfix({"손실": total_loss / num_batches})

    # 에포크 메트릭 표시
    avg_loss = total_loss / num_batches
    test_acc = calculate_accuracy(model, test_loader)
    print(f"평균 손실: {avg_loss:.4f}, 테스트 정확도: {test_acc:.4f}")

# 미세 튜닝된 모델 저장
model.save_pretrained("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")

# 모델 테스트
test_input = "I'm so happy to be able to finetune an LLM!"
test_model("./finetuned_model", test_input)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

평균 손실: 0.0851, 테스트 정확도: 0.9395


에포크 7/8: 100%|██████████| 1125/1125 [00:53<00:00, 20.85it/s, 손실=0.0806]


평균 손실: 0.0806, 테스트 정확도: 0.9400


에포크 8/8: 100%|██████████| 1125/1125 [00:54<00:00, 20.78it/s, 손실=0.0767]


평균 손실: 0.0767, 테스트 정확도: 0.9460
입력: I'm so happy to be able to finetune an LLM!
예측된 감정: joy
